In [1]:
import pandas as pd
from sklearn import model_selection, preprocessing
from sklearn import tree, linear_model, svm, ensemble, metrics
from matplotlib import pyplot as plt
import xgboost as xgb
import scipy
import numpy as np

import compare_y
import cross_validation_result as cvr
import ML_helper

%matplotlib inline

In [2]:
# load PSL linear and random results
TRUTH_linear = "../psl/data/simulation/linear/seed0/cross_val_6fold/"
INFER_linear = "../psl/result/simulation/linear/"
linear_df = cvr.cross_val_result(TRUTH_linear, INFER_linear)

# load PSL random results
TRUTH_random = "../psl/data/simulation/random/seed0/cross_val_6fold/"
INFER_random = "../psl/result/simulation/random/"
random_df = cvr.cross_val_result(TRUTH_random, INFER_random)


In [2]:
Classifiers = {"Random Forest": ensemble.RandomForestRegressor(),
               "SVM": svm.SVR(kernel="rbf"), 
               "XGBoost": xgb.XGBRegressor(n_estimators=200),
               "Linear": linear_model.LinearRegression()}

In [16]:
X = np.random.rand(100,2)
Y = X[:,0]*2 + X[:,1]

# poly = preprocessing.PolynomialFeatures(2)
# X = poly.fit_transform(X)

mse, rho = ML_helper.run_cross_val(X, Y, Classifiers)

print mse
print rho

    XGBoost       SVM  Random Forest        Linear
1  0.014183  0.005085       0.021812  5.508160e-32
2  0.006126  0.002952       0.006740  1.321188e-31
3  0.003362  0.002408       0.004676  4.314083e-32
4  0.010796  0.007578       0.016240  1.961078e-31
5  0.015273  0.001542       0.012223  8.705203e-32
    XGBoost       SVM  Random Forest  Linear
1  0.976471  0.997059       0.979412     1.0
2  1.000000  0.997059       0.994118     1.0
3  0.985294  1.000000       0.988235     1.0
4  0.994118  0.994118       0.994118     1.0
5  0.979412  1.000000       1.000000     1.0


In [39]:
X1, linear_Y = ML_helper.load_data("../data/similuated_matrix_linear.tsv")
X2, random_Y = ML_helper.load_data("../data/similuated_matrix_random.tsv")

poly = preprocessing.PolynomialFeatures(2)
X1 = poly.fit_transform(X1)
X2 = poly.fit_transform(X2)




In [40]:
mse1, rho1 = ML_helper.run_cross_val(X1, linear_Y, Classifiers, n_fold=6)
print mse1.mean()
print rho1.mean()

mse2, rho2 = ML_helper.run_cross_val(X2, random_Y, Classifiers, n_fold=6)
print mse2.mean()
print rho2.mean()


XGBoost          3.116936e-03
SVM              5.475932e-02
Random Forest    8.035053e-03
Linear           2.948723e-30
dtype: float64
XGBoost          0.984649
SVM              0.579088
Random Forest    0.947932
Linear           1.000000
dtype: float64
XGBoost          0.103438
SVM              0.091092
Random Forest    0.107551
Linear           0.128691
dtype: float64
XGBoost         -0.060978
SVM             -0.026472
Random Forest   -0.051910
Linear          -0.078711
dtype: float64
